In [1]:
import numpy as np 
import pandas as pd
import glob
import os

from matplotlib import pyplot as plt, pylab as pl
%matplotlib inline
plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (12,4)
import seaborn as sns
import plotly.express as px

#loading:
from base64 import b64decode, b64encode
from gzip import decompress, compress
import json

import pyarrow.parquet as pq
import pyarrow as pa

from datetime import datetime

from scipy.stats import skew, kurtosis
from sklearn.linear_model import LinearRegression

In [2]:
# Vectors and Metrics
PT_LARGE_SYN = 'PT_LARGE_SYN'
PT_SYN = 'PT_SYN'
PT_TCP = 'PT_TCP'
PT_DNS = 'PT_DNS'
PT_DNS_RESPONSE = 'PT_DNS_RESPONSE'
PT_NTP = 'PT_NTP'
PT_SSDP = 'PT_SSDP'
PT_ICMP = 'PT_ICMP'
PT_GENERAL = 'PT_GENERAL'
PT_TOTAL = 'PT_TOTAL'
PT_UDP = 'PT_UDP'
PT_FRAG = 'PT_FRAG'
PT_NETFLOW = 'PT_NETFLOW'

PPS = 'PPS'
BW = 'BW'

vectors = [PT_LARGE_SYN, PT_SYN,PT_TCP, PT_DNS, PT_DNS_RESPONSE, PT_NTP,PT_SSDP,PT_ICMP,PT_GENERAL,PT_TOTAL,PT_UDP,PT_FRAG, PT_NETFLOW]
metrics = [PPS,BW]

In [3]:
VECTOR = 'PT_TCP'
METRIC = 'PPS'

In [4]:
def calc_features_from_ts(ts):
        '''Calculating Features from time series of IP'''
        new_row = []
        
        df_daily_max = ts.resample('D').max()
        df_daily_mean = ts.resample('D').mean()
        df_daily_median = ts.resample('D').median()
        df_daily_q90 = ts.resample('D').quantile(0.9)


        new_row.append(df_daily_max['passed_val'].quantile(0.90))

        new_row.append(df_daily_mean['passed_val'].quantile(0.90))

        new_row.append(df_daily_median['passed_val'].quantile(0.90))

        new_row.append(df_daily_q90['passed_val'].quantile(0.90))
        

        # quantile 70
        new_row.append(df_daily_max['passed_val'].quantile(0.70))

        new_row.append(df_daily_mean['passed_val'].quantile(0.70))

        new_row.append(df_daily_median['passed_val'].quantile(0.70))

        new_row.append(df_daily_q90['passed_val'].quantile(0.70))

        # top 4 maximums
        df_daily_max_sorted = df_daily_max.sort_values(by=['passed_val'], ascending=False)

        if df_daily_max_sorted['passed_val'].shape[0] > 0:
                new_row.append(df_daily_max_sorted['passed_val'][0])
        else:
                new_row.append(np.nan)
        

        if df_daily_max_sorted['passed_val'].shape[0] > 1:
                new_row.append(df_daily_max_sorted['passed_val'][1])
        else:
                new_row.append(np.nan)
        

        if df_daily_max_sorted['passed_val'].shape[0] > 2:
                new_row.append(df_daily_max_sorted['passed_val'][2])    
        else:
                new_row.append(np.nan)
        

        if df_daily_max_sorted['passed_val'].shape[0] > 3:
                new_row.append(df_daily_max_sorted['passed_val'][3])
        else:
                new_row.append(np.nan)


        return new_row

In [5]:
def extract_ts_from_dict(ts_dict):
    '''Creating the time series data to be later added to a file'''
    passed_values = []
    blocked_values = []

    for day, values in ts_dict['passedDays'].items():
        for value in values['values']:
            passed_values.append(value)

    for day, values in ts_dict['blockedDays'].items():
        for value in values['values']:
            blocked_values.append(value)

    # Some times passed_Values or blocked values are empty
    if len(passed_values) == 0:
        passed = pd.DataFrame({'passed_val':[],'passed_tmstmp':[]})
    else:    
        passed = pd.DataFrame(passed_values).rename(columns={'value': 'passed_val', 'timeStamp': 'passed_tmstmp'})
    
    if len(blocked_values) == 0:
        blocked = pd.DataFrame({'blocked_val':[],'blocked_tmstmp':[]})
    else:    
        blocked = pd.DataFrame(blocked_values).rename(columns={'value': 'blocked_val', 'timeStamp': 'blocked_tmstmp'})

    #display(blocked.blocked_val.sum())
    ts = pd.concat([passed, blocked], axis=1).drop(['blocked_tmstmp'], axis = 1)  
    ts.set_index(['passed_tmstmp'], inplace=True, drop=True, append=False, verify_integrity=False   ) 
    ts.index = pd.to_datetime(ts.index, unit = 's')
    
    ts = ts.sort_index()
    ts.fillna(0, inplace=True)
    ts['total_val'] = ts['passed_val'] + ts['blocked_val']

    return ts

In [6]:
# 

def calc_features(df, plot = False):
        
        daily_max_q90 = []
        daily_mean_q90 = []
        daily_median_q90 = []
        daily_q90_q90 = []

        daily_max_q70 = []
        daily_mean_q70 = []
        daily_median_q70 = []
        daily_q90_q70 = []

        daily_max_1 = []
        daily_max_2 = []
        daily_max_3 = []
        daily_max_4 = []



        for _, row in df.iterrows():

                pred_id = row['prediction_id']
                parquet_file = row['ts_name']

                ts_data_raw = pd.read_parquet(directory.joinpath(parquet_file + '.parquet'), engine='pyarrow').sort_index()

                        # Filter out blocked traffic:
                ts_data = ts_data_raw[~(ts_data_raw['blocked_val'] > 0)]
        
                        # A variety of resamples: 
                df_daily_max = ts_data.resample('D').max()
                df_daily_mean = ts_data.resample('D').mean()
                df_daily_median = ts_data.resample('D').median()
                df_daily_q90 = ts_data.resample('D').quantile(0.9)

                        # Calculate features:
                # quantile 90                
                daily_max_q90.append(df_daily_max['passed_val'].quantile(0.90))
                daily_mean_q90.append(df_daily_mean['passed_val'].quantile(0.90))
                daily_median_q90.append(df_daily_median['passed_val'].quantile(0.90))
                daily_q90_q90.append(df_daily_q90['passed_val'].quantile(0.90))

                # quantile 70
                daily_max_q70.append(df_daily_max['passed_val'].quantile(0.70))
                daily_mean_q70.append(df_daily_mean['passed_val'].quantile(0.70))
                daily_median_q70.append(df_daily_median['passed_val'].quantile(0.70))
                daily_q90_q70.append(df_daily_q90['passed_val'].quantile(0.70))

                # top 4 maximums
                df_daily_max_sorted = df_daily_max.sort_values(by=['passed_val'], ascending=False)
                daily_max_1.append(df_daily_max_sorted['passed_val'][0])
                daily_max_2.append(df_daily_max_sorted['passed_val'][1])
                daily_max_3.append(df_daily_max_sorted['passed_val'][2])
                daily_max_4.append(df_daily_max_sorted['passed_val'][3])

                # Plot
                if plot == True:
                        fig = go.Figure()
                        # Add raw data plot
                        fig.add_trace(go.Scatter(x=ts_data_raw.index, y=ts_data_raw['passed_val'],  mode='lines', name='ts_data_raw'))
                        # Add filtered from blocked values plot
                        fig.add_trace(go.Scatter(x=ts_data_raw.index, y=ts_data['passed_val'],  mode='markers', name='ts_data'))

                        # Add resampled data plot
                        fig.add_trace(go.Scatter(x=df_daily_max.index, y=df_daily_max['passed_val'],  mode='markers', name='daily_max'))
                        fig.add_trace(go.Scatter(x=df_daily_q90.index, y=df_daily_q90['passed_val'],  mode='markers', name='daily_q90'))

                        # plot features:
                        fig.add_trace(go.Scatter(x=ts_data_raw.index, y=[daily_max_q90]*len(ts_data_raw), mode='lines', name='daily_max_q90'))
                        fig.add_trace(go.Scatter(x=ts_data_raw.index, y=[daily_max_q70]*len(ts_data_raw), mode='lines', name='daily_max_q70'))
                        fig.add_trace(go.Scatter(x=ts_data_raw.index, y=[3*daily_max_q90]*len(ts_data_raw), mode='lines', name='3*daily_max_q90'))

                        # plot thresholds:
                        fig.add_trace(go.Scatter(x=ts_data_raw.index, y=[row['ip_ss']]*len(ts_data_raw), mode='lines', line=dict(dash='dash'), name='ip_ss'))
                        fig.show()

        df['daily_max_q90'] = daily_max_q90
        df['daily_mean_q90'] = daily_mean_q90
        df['daily_median_q90'] = daily_median_q90
        df['daily_q90_q90'] = daily_q90_q90

        df['daily_max_q70'] = daily_max_q70
        df['daily_mean_q70'] = daily_mean_q70
        df['daily_median_q70'] = daily_median_q70
        df['daily_q90_q70'] = daily_q90_q70

        df['daily_max_1'] = daily_max_1
        df['daily_max_2'] = daily_max_2
        df['daily_max_3'] = daily_max_3
        df['daily_max_4'] = daily_max_4

        return df

In [7]:
tcp_pps = pd.read_csv('./tcp_pps_v7.csv').drop(columns=['Unnamed: 0']) # 1787 rows
tcp_pps

,prediction_id,vector,metric,file,super_peaks_file,ip_ss,daily_max_q90,daily_mean_q90,daily_median_q90,daily_q90_q90,daily_max_q70,daily_mean_q70,daily_median_q70,daily_q90_q70,daily_max_1,daily_max_2,daily_max_3,daily_max_4
0,a85101f9a44c3e7fa26ad3a8f5258b1f6b780aed7e1c38...,PT_TCP,PPS,./../../Itay_&_Mila_data/all_vec_met/PT_TCP_PP...,NaN,12.0,0.32800,0.013707,0.012000,0.026000,0.22200,0.013174,0.011000,0.02400,0.36100,0.34400,0.33900,0.32800
1,f6ab823e547c7ec805b878d2250509cb810addb988ce57...,PT_TCP,PPS,./../../Itay_&_Mila_data/all_vec_met/PT_TCP_PP...,NaN,30.0,15.46400,1.438101,0.968000,2.950200,14.94100,1.233787,0.740000,2.75980,17.05900,16.32800,15.79300,15.46400
2,ac5b3676bb10fc8ae4cd494e3724053321f533b28c3a64...,PT_TCP,PPS,./../../Itay_&_Mila_data/all_vec_met/PT_TCP_PP...,./../../Itay_&_Mila_data/super_peaks/PT_TCP_PP...,65.0,36.82500,10.827847,10.168050,17.704560,24.30600,9.568966,8.899950,15.38814,105.93900,70.05795,61.31595,36.82500
3,3ca2d2ca5e29360360beef068d732605d4d4499209cdf3...,PT_TCP,PPS,./../../Itay_&_Mila_data/all_vec_met/PT_TCP_PP...,./../../Itay_&_Mila_data/super_peaks/PT_TCP_PP...,68.0,64.64205,26.075037,28.985475,44.062260,59.09805,23.979144,25.009050,41.58135,75.12705,64.87200,64.67205,64.64205
4,4446402910b942444736abdc234a69c1be8cab947550b2...,PT_TCP,PPS,./../../Itay_&_Mila_data/all_vec_met/PT_TCP_PP...,./../../Itay_&_Mila_data/super_peaks/PT_TCP_PP...,41.0,24.86205,5.571671,5.067000,11.071395,19.12395,5.403210,4.541475,10.58505,36.03105,29.05905,25.01400,24.86205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,79296fdcd9d94f6a606db159963999bfc33cb8698ba71f...,PT_TCP,PPS,./../../Itay_&_Mila_data/all_vec_met/PT_TCP_PP...,./../../Itay_&_Mila_data/super_peaks/PT_TCP_PP...,45.0,38.55000,11.764190,6.783975,26.809650,30.18405,9.537494,5.131500,23.04000,98.92905,81.83805,46.54095,38.55000
1783,161e1a1669916c2486a5d2debc2f03ae43052b01c062c6...,PT_TCP,PPS,./../../Itay_&_Mila_data/all_vec_met/PT_TCP_PP...,./../../Itay_&_Mila_data/super_peaks/PT_TCP_PP...,32.0,19.74795,1.831723,1.050975,4.537740,6.05805,0.227902,0.031950,0.67560,32.27895,27.07005,24.79005,19.74795
1784,99b741be8b52a7ff1e704abf80d85c000b0d9d33feedc3...,PT_TCP,PPS,./../../Itay_&_Mila_data/all_vec_met/PT_TCP_PP...,./../../Itay_&_Mila_data/super_peaks/PT_TCP_PP...,87.0,66.82305,2.900799,0.822000,8.486760,51.13200,2.485531,0.711975,7.73400,79.20600,75.39000,67.31895,66.82305
1785,84bee7e6f334fb45dfa5b2089e06cece732e898ad7e1a2...,PT_TCP,PPS,./../../Itay_&_Mila_data/all_vec_met/PT_TCP_PP...,./../../Itay_&_Mila_data/super_peaks/PT_TCP_PP...,16.7,9.73395,2.926451,2.955975,4.491300,9.43005,2.748698,2.685450,4.35216,10.27800,10.22805,9.76500,9.73395


### Testing different blocked val threshold:

In [8]:

# data = pd.read_csv('./../../Itay_&_Mila_data/all_vector_metric_prediction_id-combinations/FileCatalog.csv')
# data

In [9]:
data_v2_p1 = pd.read_csv('./FileCatalog_v2_p1.csv')
data_v2_p2 = pd.read_csv('./FileCatalog_v2_p2.csv')
data = pd.concat([data_v2_p1, data_v2_p2])
data


C:\Users\itay8\AppData\Local\Temp\ipykernel_9028\774743574.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_v2_p1 = pd.read_csv('./FileCatalog_v2_p1.csv')
C:\Users\itay8\AppData\Local\Temp\ipykernel_9028\774743574.py:2: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_v2_p2 = pd.read_csv('./FileCatalog_v2_p2.csv')


,id,prediction_id,original_file,vector,metric,request_id,ip_ss,ip_sc,ip_nw,ip_rl,...,total_val_slope,total_val_crest_factor,total_val_shape_factor,total_val_avg_first_order_diff,missing_tmstmp_percentage,percent_of_largest_dead,percent_of_zeros,total_time,total_val_median_share,super_peaks_file
0,4722,a85101f9a44c3e7fa26ad3a8f5258b1f6b780aed7e1c38...,TimeSeriesDataB64_0_0_100.parquet,PT_DNS_RESPONSE,BW,395,2.0,4.0,6.0,9.0,...,2.181106e-10,40.211269,8.916736,2.668500e-24,0.069444,0.338,0.976257,30.0,0.000000,NaN
1,4716,a85101f9a44c3e7fa26ad3a8f5258b1f6b780aed7e1c38...,TimeSeriesDataB64_0_0_100.parquet,PT_LARGE_SYN,BW,395,0.1,0.2,0.2,0.2,...,-2.288744e-11,123.812391,46.670364,5.023059e-24,0.069444,0.240,0.999166,30.0,0.000000,NaN
2,4730,a85101f9a44c3e7fa26ad3a8f5258b1f6b780aed7e1c38...,TimeSeriesDataB64_0_0_100.parquet,PT_UDP,BW,395,7.0,12.0,16.0,30.0,...,4.463537e-08,162.524787,10.914687,-5.258403e-09,0.069444,0.000,0.001181,30.0,0.008846,NaN
3,4720,a85101f9a44c3e7fa26ad3a8f5258b1f6b780aed7e1c38...,TimeSeriesDataB64_0_0_100.parquet,PT_DNS,BW,395,1.0,2.0,3.0,4.0,...,5.696970e-09,24.139679,13.466244,-4.077000e-09,0.069444,0.001,0.497081,30.0,0.000821,NaN
4,4732,a85101f9a44c3e7fa26ad3a8f5258b1f6b780aed7e1c38...,TimeSeriesDataB64_0_0_100.parquet,PT_ICMP,BW,395,5.0,7.5,10.0,20.0,...,3.013283e-10,31.148846,2.104985,-1.482545e-09,0.069444,0.000,0.000023,30.0,0.004885,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40977,99635,a081170b6a3b5a899dc186b563488c5edd626214a9f310...,TimeSeriesDataB64_4_900_1000.parquet,PT_LARGE_SYN,PPS,4941,0.4,0.6,0.6,0.6,...,7.913066e-11,66.776758,29.192142,2.208907e-24,27.299872,0.080,0.996708,30.0,0.000000,NaN
40978,99629,a081170b6a3b5a899dc186b563488c5edd626214a9f310...,TimeSeriesDataB64_4_900_1000.parquet,PT_ICMP,PPS,4941,5.0,7.0,9.0,15.0,...,-2.653187e-10,23.467987,5.881327,7.639066e-09,0.000000,0.000,0.240486,30.0,0.000164,NaN
40979,99631,a081170b6a3b5a899dc186b563488c5edd626214a9f310...,TimeSeriesDataB64_4_900_1000.parquet,PT_SYN,PPS,4941,10.0,12.0,18.0,20.0,...,-9.371713e-07,71.943775,1.532923,-3.078775e-08,0.000000,NaN,0.000000,30.0,0.054190,NaN
40980,99641,a081170b6a3b5a899dc186b563488c5edd626214a9f310...,TimeSeriesDataB64_4_900_1000.parquet,PT_SSDP,PPS,4941,0.4,0.5,0.7,0.7,...,-9.576488e-10,37.016018,3.255654,4.185067e-22,0.000000,0.000,0.059306,30.0,0.000304,NaN


In [10]:
# def filter_data_frame(df, **kwargs,):
#     filtered_df = df.copy()
#     for col, val in kwargs.items():
#         if isinstance(val, list):
#             filtered_df = filtered_df[filtered_df[col].isin(val)]
#         else:
#             filtered_df = filtered_df[filtered_df[col] == val]
#     return filtered_df

In [11]:
# filtered_df = filter_data_frame(data,vector=VECTOR, metric=METRIC,)
# filtered_df

In [13]:
# def get_row_info(row):
#     res = {
#         'metric':row['metric'],
#         'vector':row['vector'],
#         'pred_id':row['prediction_id'],
#         'ip_ss':row['ip_ss'],
#         'ip_sc':row['ip_sc'],
#         'ip_nw':row['ip_nw'],
#         'ip_rl':row['ip_rl'],
#         'range_ss':row['range_ss'],
#         'range_sc':row['range_sc'],
#         'range_nw':row['range_nw'],
#         'range_rl':row['range_rl'],
#         'start_time':row['start_time'],
#         'end_time':row['end_time'],
#     }
#     return res

In [14]:
# def get_ts_by_vec_and_metric_filtered(df):
#     '''Returns times series with value and time stamp'''
#     file_list = df.file.values
#     results = []
#     for i,row in df.iterrows():
#         row_info = get_row_info(row)
#         file = row['file']
#         ts_hash = pd.read_parquet(file).time_series.values[0]
#         ts_dict = json.loads(decompress(b64decode(ts_hash)))
        
#     #print(ts_dict['underAttack'], vec, metric)
#         passed_values = []
#         blocked_values = []

#         for day, values in ts_dict['passedDays'].items():
#             for value in values['values']:
#                 passed_values.append(value)

#         for day, values in ts_dict['blockedDays'].items():
#             for value in values['values']:
#                 blocked_values.append(value)

#         # Some times passed_Values or blocked values are empty
#         if len(passed_values) == 0:
#             passed = pd.DataFrame({'passed_val':[],'passed_tmstmp':[]})
#         else:    
#             passed = pd.DataFrame(passed_values).rename(columns={'value': 'passed_val', 'timeStamp': 'passed_tmstmp'})
        
#         if len(blocked_values) == 0:
#             blocked = pd.DataFrame({'blocked_val':[],'blocked_tmstmp':[]})
#         else:    
#             blocked = pd.DataFrame(blocked_values).rename(columns={'value': 'blocked_val', 'timeStamp': 'blocked_tmstmp'})

#         #display(blocked.blocked_val.sum())
#         curr_res = pd.concat([passed, blocked], axis=1).drop(['blocked_tmstmp'], axis = 1)  
#         curr_res.set_index(['passed_tmstmp'], inplace=True) 
#         curr_res.index = pd.to_datetime(curr_res.index, unit = 's')
#         curr_res.sort_index()
#         curr_res.fillna(0, inplace=True)
#         curr_res['total_val'] = curr_res['passed_val'] + curr_res['blocked_val']

#         results.append((row_info, curr_res))

#     return results

In [15]:
# filtered_data = get_ts_by_vec_and_metric_filtered(filtered_df)
# filtered_data

In [16]:
def compute_features(df):
    def compute_column_features(col):
        mean = col.mean()
        median = col.median()
        mode = col.mode()[0]
        variance = col.var()
        std_dev = col.std()
        skewness = skew(col)
        kurt = kurtosis(col)
        iqr = col.quantile(0.75) - col.quantile(0.25)
        mad = col.mad()
        
        rms = np.sqrt(np.mean(col**2))

        
            
        median_crossing_rate = np.sum(np.diff(np.sign(col - col.median())) != 0) / (len(col) - 1)
        x = np.arange(len(col)).reshape(-1, 1)
        y = col.values.reshape(-1, 1)
        slope = LinearRegression().fit(x, y).coef_[0][0]
        max_abs_val = col.abs().max()

        # Check if the RMS value is zero
        if rms == 0:
            crest_factor = np.nan  # Assign NaN or any other appropriate value to the Crest Factor
        else:
            crest_factor = max_abs_val / rms
        
        mean_abs_val = col.abs().mean()
        if mean_abs_val == 0:
            shape_factor = np.nan  # Assign NaN or any other appropriate value to the Crest Factor
        else:
            shape_factor = rms / mean_abs_val

        avg_first_order_diff = np.mean(np.diff(col))

        return {
            'mean': mean,
            'median': median,
            'mode': mode,
            'variance': variance,
            'std_dev': std_dev,
            'skewness': skewness,
            'kurtosis': kurt,
            'iqr': iqr,
            'mad': mad,
            'rms': rms,
            'median_crossing_rate': median_crossing_rate,
            'slope': slope,
            'crest_factor': crest_factor,
            'shape_factor': shape_factor,
            'avg_first_order_diff': avg_first_order_diff
        }

    features = {
        'passed_val': compute_column_features(df['passed_val']),
        'blocked_val': compute_column_features(df['blocked_val']),
        'total_val': compute_column_features(df['total_val'])
    }

    return features


In [17]:
# def add_features_to_df(tpl, target_df):
#     # Extract the dictionary and time series DataFrame from the tuple
#     d, ts = tpl

#     # Compute the features using the updated compute_features() function
#     features = compute_features(ts)

#     # Find the index in target_df that matches the prediction_id, vector, and metric
#     index = target_df.loc[
#         (target_df['prediction_id'] == d['pred_id']) &
#         (target_df['vector'] == d['vector']) &
#         (target_df['metric'] == d['metric'])
#     ].index

#     # Check if there is a single matching row
#     if len(index) == 1:
#         # Add the computed features to the matching row in target_df
#         for col_name, col_features in features.items():
#             for feature_name, feature_value in col_features.items():
#                 target_df.at[index[0], f"{col_name}_{feature_name}"] = feature_value
#     else:
#         print(f"Error: Found {len(index)} matching rows for prediction_id={d['pred_id']}")


In [18]:
# for vec in vectors:
#     for met in metrics:
#         filtered_df = filter_data_frame(data,vector=vec, metric=met)
#         filtered_data = get_ts_by_vec_and_metric_filtered(filtered_df)
#         print('Create filtered_data for', vec, met)
#         for fd in filtered_data:
#             add_features_to_df(fd, data_cp)
#         print('added features data for', vec, met)

In [19]:
# data_cp.to_csv('./../../Itay_&_Mila_data/all_vector_metric_prediction_id-combinations/FileCatalog_improved.csv')
# data_cp.to_csv('./FileCatalog_improved.csv')

In [20]:
# data_improved = pd.read_csv('./FileCatalog_improved.csv')
# data_improved

### Recreating the parquet files

In [21]:
# # Creating the parquet files with the time series data frame
# for vec in vectors:
#     for met in metrics:
#         filtered_df = filter_data_frame(data_improved,vector=vec, metric=met)
#         print('filtered by vec and met')
#         filtered_data = get_ts_by_vec_and_metric_filtered(filtered_df)
#         print('created dic for ', vec, met)
#         for fd in filtered_data:
#             df = fd[1]
#             vec_met = vec + '_' + met
#             new_file_name = './../../Itay_&_Mila_data/all_vec_met/' +vec_met+'/' + vec_met + '_' + fd[0]['pred_id']
#             df.to_parquet(new_file_name)